In [ ]:
import Pkg;
Pkg.add("ReinforcementLearning")
Pkg.add("ReinforcementLearningBase")
Pkg.add("ReinforcementLearningCore")
Pkg.add("CUDA")
Pkg.add("CompilerPluginTools")
Pkg.add("PlotlyJS")
Pkg.add("DifferentialEquations")
Pkg.add("Plots")
Pkg.add("Flux")
Pkg.add("Distributions")
Pkg.instantiate()

In [1]:
using DifferentialEquations
using Plots
using ReinforcementLearningBase
using ReinforcementLearningCore
using Random
using LinearAlgebra
using Flux
using Distributions
using Flux: params, gradient
using CUDA

In [2]:
CUDA.device()

CuDevice(0): NVIDIA GeForce RTX 3080

In [3]:
mass = 1.0
force_per_rpm = 4e-10
distance_to_propellers = 0.15
moment_of_inertia = mass * distance_to_propellers^2


0.0225

In [4]:
struct ContinuousDroneActionSpace
    action_range::Tuple{CuArray{Float64}, CuArray{Float64}} # define action_range within the space struct
end

In [5]:
mutable struct ContinuousDroneStateSpace
	position_low::CuArray{Float64,1}
	position_high::CuArray{Float64,1}
	velocity_low::CuArray{Float64,1}
	velocity_high::CuArray{Float64,1}
	orientation_low::CuArray{Float64,1}
	orientation_high::CuArray{Float64,1}
	angular_velocity_low::CuArray{Float64,1}
	angular_velocity_high::CuArray{Float64,1}
end

In [6]:
import Base: rand

function rand(rng::AbstractRNG, space::ContinuousDroneActionSpace, num::Int)
    low, high = space.action_range
    return [rand(rng, low[i]:high[i]) for i in 1:length(low), _ in 1:num]
end

import Random

function Random.Sampler(rng::Random.AbstractRNG, space::ContinuousDroneActionSpace, ::Random.Repetition)
    return space
end

function Random.rand(rng::Random.AbstractRNG, sampler::ContinuousDroneActionSpace)
    low, high = sampler.action_range
    return [rand(rng, low[i]:high[i]) for i in 1:length(low)]
end

In [7]:
mutable struct Drone
	position::CuArray{Float64,1}
	velocity::CuArray{Float64,1}
	orientation::CuArray{Float64,1} # roll, pitch, yaw in radians
	angular_velocity::CuArray{Float64,1}

	mass::Float64
	force_per_rpm::Float64
	distance_to_propellers::Float64
	moment_of_inertia::Float64
end

In [8]:
mutable struct DroneEnv <: AbstractEnv
    drone::Drone
    target_position::CuArray{Float64,1}
    previous_position::CuArray{Float64,1}
    action::CuArray{Float64,1}
    reward::Float64
    done::Bool

    velocity_threshold::Float64
    alititude_threshold::Float64
    angular_velocity_threshold::Float64
    roll_threshold::Float64
    pitch_threshold::Float64

    time::Float64
    max_time::Float64
end

function DroneEnv()
    drone = Drone(zeros(3), zeros(3), zeros(3), zeros(3), mass, force_per_rpm, distance_to_propellers, moment_of_inertia)
    action = zeros(4)
    target_position = [10.0, 10.0, 10.0]
    previous_position = zeros(3)
    DroneEnv(drone, target_position, previous_position, action, 0.0, false, 10, 10, 5, pi/4, pi/4, 0, 100)
end

function DroneEnv(target_position::CuArray{Float64,1})
    drone = Drone(zeros(3), zeros(3), zeros(3), zeros(3), mass, force_per_rpm, distance_to_propellers, moment_of_inertia)
    action = zeros(4)
    previous_position = zeros(3)
    DroneEnv(drone, target_position, previous_position, action, 0.0, false, 10, 10, 5, pi/4, pi/4, 0, 100)
end

DroneEnv

In [9]:
### Define Action Space for quadcopter rpm of each motor
RLBase.action_space(env::DroneEnv) = ContinuousDroneActionSpace((zeros(4), ones(4)*20000))
RLBase.state(env::DroneEnv) = [env.drone.position; env.drone.velocity; env.drone.orientation; env.drone.angular_velocity; env.drone.mass; env.drone.force_per_rpm; env.drone.distance_to_propellers; env.drone.moment_of_inertia]
# RLBase.is_terminated(env::DroneEnv) = env.done
RLBase.state_space(env::DroneEnv) = ContinuousDroneStateSpace(
	[-100.0, -10.0, 0.0],
	[100.0, 100.0, 100.0], 
	[-10.0, -10.0, -10.0],
	[10.0, 10.0, 10.0],
	[-pi, -pi, -pi],
	[pi/1, pi/1, pi/1],
	[-pi, -pi, -pi],
	[pi/1, pi/1, pi/1],
)
# RLBase.action_space(env::DroneEnv, action::CuArray{Float64,1}) = ContinuousDroneActionSpace(zeros(4), ones(4)*2000)

In [10]:
function RLBase.reset!(env::DroneEnv)
    drone = Drone(zeros(3), zeros(3), zeros(3), zeros(3), mass, force_per_rpm, distance_to_propellers, moment_of_inertia)
    env.previous_position = zeros(3)
    env.done = false
    env.reward = 0.0
    env.time = 0.0
    env.drone = drone
end

In [11]:
function drone_dynamics!(du, u, p, t)
    gravity = 9.81 # gravitational constant, m/s^2
    rpms = p(t) # propeller speeds in RPM at time t
    
    ### Get the mass, force_per_rpm, distance_to_propellers, and moment_of_inertia from the drone
    mass = u[13]
    force_per_rpm = u[14]
    distance_to_propellers = u[15]
    moment_of_inertia = u[16]


    ### force generated by each propeller is squared to account for the fact that the force is proportional to the square of the propeller speed
    forces = [force_per_rpm*(rpms[1]^2), force_per_rpm*(rpms[2]^2), force_per_rpm*(rpms[3]^2), force_per_rpm*(rpms[4]^2)] # force generated by each propeller
    net_force = sum(forces) - mass*gravity # net force (upward force - weight)
    net_torque_pitch = (forces[1] + forces[4] - forces[2] - forces[3])*distance_to_propellers # net torque around x-axis
    net_torque_roll = (forces[1] + forces[2] - forces[3] - forces[4])*distance_to_propellers # net torque around y-axis
    net_torque_yaw = (forces[1] + forces[3] - forces[2] - forces[4])*distance_to_propellers # net torque around z-axis


    du[1:3] = u[4:6] # position updates with velocity

    # Update the x and z velocity and position based on pitch roll and yaw torques
    # du[4] = net_t

    # du[1] = du[1]*cos(u[6]) - du[2]*sin(u[6])
    # du[2] = du[1]*sin(u[6]) + du[2]*cos(u[6])

    du[4:6] = [u[7]*cos(u[6]) - u[8]*sin(u[6]), u[7]*sin(u[6]) + u[8]*cos(u[6]), net_force/mass] # velocity updates with acceleration

    # Check if drone is on the ground
    if du[3] < 0.0
        du[3] = 0.0
        du[6] = 0.0
    end

    # Check if drone is upside down
    if u[3] > pi/2 || u[3] < -pi/2
        du[4:6] = [0.0, 0.0, 0.0]
    end

    du[7:9] = u[10:12] # orientation updates with angular velocity
    du[10:12] = [net_torque_roll/moment_of_inertia, net_torque_pitch/moment_of_inertia, net_torque_yaw/moment_of_inertia] # angular velocity updates with angular acceleration
end

drone_dynamics! (generic function with 1 method)

In [12]:
# function RewardFunction(env::DroneEnv)
# 	tot_reward = 0.0
# 	prev_dist_to_target = norm(env.previous_position - env.target_position)
# 	dist_to_target = norm(env.drone.position - env.target_position)
# 	yaw = env.drone.orientation[3]
# 	roll = env.drone.orientation[1]
# 	pitch = env.drone.orientation[2]

# 	tot_reward -= dist_to_target

# 	if roll > pi/2 || roll < -pi/2 || pitch > pi/4 || pitch < -pi/4
# 		tot_reward -= 1000
# 	end

# 	facing_dir = [cos(yaw), sin(yaw)]
# 	drone_dir = [cos(roll)*cos(pitch), sin(roll)*cos(pitch)]
# 	if dot(facing_dir, drone_dir) < 0.0
# 		tot_reward -= 1000
# 	end

# 	if dot(env.drone.velocity, env.target_position - env.drone.position) < 0.0
# 		tot_reward -= 1000
# 	end
	
# 	if env.drone.angular_velocity[1] != 0.0 || env.drone.angular_velocity[2] != 0.0
# 		tot_reward -= abs(sum(env.drone.angular_velocity[1:2]))
# 	end

# 	if env.drone.angular_velocity[3] != 0.0
# 		tot_reward -= abs(env.drone.angular_velocity[3])
# 	end

# 	if dist_to_target < prev_dist_to_target
# 		tot_reward += 100
# 	end
# 	if dist_to_target < 0.1
# 		tot_reward += 1000
# 	end

# 	return tot_reward
# end

function RewardFunction(env::DroneEnv)
    # Compute the distance to the target
    distance_to_target = norm(env.drone.position - env.target_position)

    # Compute the energy cost
    energy_cost = sum(env.action)

    # Reward is higher for smaller distances to target and lower energy costs
    reward = -distance_to_target - 0.01 * energy_cost

    return reward
end

RewardFunction (generic function with 1 method)

In [13]:
function RLBase.is_terminated(env::DroneEnv)
	### Check if the drone orientation is too far from vertical
	roll = env.drone.orientation[1]
	pitch = env.drone.orientation[2]
	if abs(roll) > env.roll_threshold || abs(pitch) > env.pitch_threshold
		# println("roll or pitch too large")
		env.done = true
	end

	if norm(env.drone.velocity) > env.velocity_threshold
		# println("velocity too large")
		env.done = true
    end

	if norm(env.drone.angular_velocity) > env.angular_velocity_threshold
		# println("angular velocity too large")
		env.done = true
	end

	if env.drone.position[3] < 0.0
		# println("drone crashed")
		env.done = true
	end

	if norm([env.drone.position[1] - env.target_position[1]; env.drone.position[2] - env.target_position[2]; env.drone.position[3] - env.target_position[3]]) < 0.1
		# println("drone reached target")		
		env.done = true
	end

	if env.time > env.max_time
		# println("time limit reached")
		env.done = true
	end

	return env.done
end

In [14]:
function step!(env::DroneEnv, action::CuArray{Float64,1, CUDA.Mem.DeviceBuffer})
	# action is an array of 4 numbers
	# action[1] is the rpm of the front left propeller
	# action[2] is the rpm of the front right propeller
	# action[3] is the rpm of the back left propeller
	# action[4] is the rpm of the back right propeller
	tspan = (0.0, 0.1) # time span for the differential equation solver
	p = (t) -> action # propeller speeds as a function of time
	env.previous_position = env.drone.position # save the previous position
	env.action = action # save the action
	u0 = [env.drone.position; env.drone.velocity; env.drone.orientation; env.drone.angular_velocity; env.drone.mass; env.drone.force_per_rpm; env.drone.distance_to_propellers; env.drone.moment_of_inertia] # initial state
	prob = ODEProblem(drone_dynamics!, u0, tspan, p) # define the ODE problem
	sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8) # solve the ODE problem
	env.drone.position = sol.u[end][1:3] # update the drone position
	env.drone.velocity = sol.u[end][4:6] # update the drone velocity
	env.drone.orientation = sol.u[end][7:9] # update the drone orientation
	env.drone.angular_velocity = sol.u[end][10:12] # update the drone angular velocity
	env.reward = RewardFunction(env) # update the reward
	env.time += 0.1 # update the time
	
	
	return env.reward, env.done
end

step! (generic function with 1 method)

In [15]:
# function get_state(drone::Drone)
#     return [drone.position..., drone.velocity..., drone.orientation..., drone.angular_velocity...]
# end

# function apply_changes!(drone::Drone, du)
#     # This function should update the drone's state based on the changes calculated by drone_dynamics!.
#     # For example:
#     drone.position .+= du[1:3]
#     drone.velocity .+= du[4:6]
#     drone.orientation .+= du[7:9]
#     drone.angular_velocity .+= du[10:12]
# end

# function ReinforcementLearningBase.step!(env::DroneEnv, action)
#     # Apply the action (change in RPMs or rotor forces), then update drone's state.
#     # Use the drone_dynamics! function here.
#     du = zeros(12) #assuming the size of du matches with the state size
#     drone_dynamics!(du, get_state(env.drone), (t) -> action, 0.0) # assuming that drone_dynamics! is defined in a way that is compatible with this use

#     # Update drone state with the changes
#     apply_changes!(env.drone, du)

#     # Also update the reward and done status.
#     env.reward = reward(env, action)
#     env.done = is_done(env)
# end

In [16]:
# env = DroneEnv()

# total_reward = 0.0
# num_steps = 0

# trajectory = []

# # Start the episode
# while !RLBase.is_terminated(env)
#     # Select a random action
#     action = rand(RLBase.action_space(env))

#     # Apply the action to the environment
#     reward, done = env(action)
# 	# println("Reward: ", reward)

#     # Update the total reward and number of steps
#     total_reward += reward
#     num_steps += 1

#     # Save the drone's position
#     push!(trajectory, env.drone.position)

#     if done
#         break
#     end
# end
# println("Episode finished after ", num_steps, " steps. Total reward: ", total_reward)
# println("Final position: ", env.drone.position)

# ### 3d Plot the trajectory
# x = [trajectory[i][1] for i in 1:length(trajectory)]
# y = [trajectory[i][2] for i in 1:length(trajectory)]
# z = [trajectory[i][3] for i in 1:length(trajectory)]

# # plotlyjs()
# plot(x, y, z, label="trajectory", xlabel="x", ylabel="y", zlabel="z", title="Drone Trajectory", legend=:bottomright)


In [17]:
# ### Start the reinforcement learning
# ### Implement a Deep learning model for Q-learning algorithm to learn the optimal policy
# ### The state space is the drone's position and velocity
# ### The action space is the change in RPMs of the propellers
# ### The reward function is the distance to the target
# ### The termination condition is when the drone reaches the target or crashes

# ### Define the Q-network
# struct QNetwork
# 	env::DroneEnv
# end

# ### Define the DeepQLearningPolicy
# struct DeepQLearningPolicy
# 	env::DroneEnv
# 	Q::QNetwork
# 	epsilon::Float64
# 	gamma::Float64
# 	alpha::Float64
# 	epsilon_decay::Float64
# 	gamma_decay::Float64
# 	alpha_decay::Float64
# 	epsilon_min::Float64
# 	gamma_min::Float64
# 	alpha_min::Float64
# end

# policy = DeepQLearningPolicy(env, QNetwork(env), 0.1, 0.99, 0.1, 0.1, 0.1, 0.1, 0.1, 0.95, 0.1)

# ### Define the Q-learning algorithm
# function QLearning(policy::DeepQLearningPolicy, num_episodes::Int64)
# 	# Initialize the Q-network
# 	Q = policy.Q
# 	# Initialize the total reward
# 	total_reward = 0.0
# 	# Initialize the number of steps
# 	num_steps = 0
# 	# Initialize the trajectory
# 	trajectory = []
# 	# Initialize the episode
# 	for i in 1:num_episodes
# 		# Initialize the environment
# 		env = policy.env
# 		# Initialize the episode
# 		while !RLBase.is_terminated(env)
# 			# Select an action
# 			action = rand(RLBase.action_space(env))
# 			# Apply the action to the environment
# 			reward, done = env(action)
# 			# Update the total reward and number of steps
# 			total_reward += reward
# 			num_steps += 1
# 			# Save the drone's position
# 			push!(trajectory, env.drone.position)
# 			# Update the Q-network
# 			Q = update(Q, env, action, reward, done)
# 			# Update the policy
# 			policy = update(policy, Q)
# 			if done
# 				break
# 			end
# 		end
# 	end
# 	return policy, Q, total_reward, num_steps, trajectory
# end

# ### Define the update function for the Q-network
# function update(Q::QNetwork, env::DroneEnv, action, reward, done)
# 	# Update the Q network based on the reward and done status
	
# end

In [18]:
### Implement the PPO algorithm
### Define the PPO policy
mutable struct PPOPolicy
	env::DroneEnv
	policy::Chain
	batch::Array{Tuple{CuArray{Float64,1}, CuArray{Float64,1}, Float64, CuArray{Float64,1}, Bool}, 1}
	epsilon::Float64
	gamma::Float64
	alpha::Float64
	epsilon_decay::Float64
	gamma_decay::Float64
	alpha_decay::Float64
	epsilon_min::Float64
	gamma_min::Float64
	alpha_min::Float64
end

function PPOPolicy(env::DroneEnv, policy::Chain, epsilon::Float64, gamma::Float64, alpha::Float64, epsilon_decay::Float64, gamma_decay::Float64, alpha_decay::Float64, epsilon_min::Float64, gamma_min::Float64, alpha_min::Float64)
	batch = [] # Empty batch to start
	return PPOPolicy(env, policy, batch, epsilon, gamma, alpha, epsilon_decay, gamma_decay, alpha_decay, epsilon_min, gamma_min, alpha_min)
end

PPOPolicy

In [19]:
function gaussian_policy(network::Chain, state::CuArray{Float64,1})
	mu, log_sigma = network(state)  # Assume the network outputs means and log standard deviations
    sigma = exp.(log_sigma)
    return MvNormal(mu, sigma)  # Return a multivariate normal distribution
end

function prob(policy::PPOPolicy, state::CuArray{Float64,1}, action::CuArray{Float64,1})
	distribution = gaussian_policy(policy.policy, state)
    return pdf(distribution, action)  # Probability density function for the given action
end

function get_action(policy::PPOPolicy, state::CuArray{Float64,1})
	mu, log_sigma = policy.policy(state)  # Assume the network outputs means and log standard deviations
    sigma = exp.(log_sigma)

	mu = Array(mu)
	sigma = Array(sigma)

	distribution = MvNormal(mu, sigma)
	action = rand(distribution)
	action = action |> gpu
	return action
end

### Define the update function for the PPO policy
function update(policy::PPOPolicy, optimizer)
	# loss = -log(prob(policy, state, action)) * reward  # Minus sign because we're maximizing reward
    # grads = gradient(() -> loss, params(policy.policy))
    # Flux.Optimise.update!(optimizer, params(policy.policy), grads)
	for (state, action, reward, new_state, done) in policy.batch
		state = state |> gpu
		action = action |> gpu

		state = convert(CuArray{Float64,1}, state)
		action = convert(CuArray{Float64,1}, action)

		action_new = get_action(policy, state)
		action_new = action_new |> gpu
		action_new = convert(CuArray{Float64,1}, action_new)

		value = reward + policy.gamma * (done ? 0.0 : prob(policy, new_state, action_new))
		advantage = value - prob(policy, state, action)
		ratio = prob(policy, state, action) / prob(policy, state, action)
		loss = -min(ratio * advantage, clamp(ratio, 1 - policy.epsilon, 1 + policy.epsilon) * advantage)
		grads = gradient(() -> loss, params(policy.policy))
		Flux.Optimise.update!(optimizer, params(policy.policy), grads)
	end
	policy.batch = []  # Empty the batch
end


### Define the PPO algorithm
function PPO(policy::PPOPolicy, num_episodes::Int64, batch_size::Int64)
	# Initialize the total reward
	total_reward = 0.0
	# Initialize the number of steps
	num_steps = 0
	# Initialize the trajectory
	trajectories = []
	optimizer = ADAM(0.01)

	# Initialize the episode
	for i in 1:num_episodes
		trajectory = []
		# Initialize the environment
		env = policy.env
		# Reset the environment
		RLBase.reset!(env)
		# Initialize the episode
		while !RLBase.is_terminated(env)
			state = RLBase.state(env)
			state = state |> gpu
			state = convert(CuArray{Float64,1}, state)

			action = get_action(policy, state)
			action = action |> gpu
			action = convert(CuArray{Float64,1}, action)


	
			# Apply the action to the environment
			reward, done = step!(env, action)
			push!(policy.batch, (state, action, reward, state, done))
			# Update the total reward and number of steps
			total_reward += reward
			num_steps += 1
			# Save the drone's position
			push!(trajectory, env.drone.position)
			# Update the policy
			if length(policy.batch) >= batch_size
				update(policy, optimizer)
			end
			if done
				break
			end
		end
		push!(trajectories, trajectory)
		# Decay the exploration
		policy.epsilon = max(policy.epsilon * policy.epsilon_decay, policy.epsilon_min)
	end
	return policy, total_reward, num_steps, trajectories
end



PPO (generic function with 1 method)

In [23]:
target_pos = CuArray([1.0, 1.0, 1.0])
env = DroneEnv(target_pos)
### Create the PPO policy# Define the policy network
policy_chain = Chain(
    Dense(16, 64, relu),
    Dense(64, 64, relu),
    Dense(64, 8),  # Assume your state is 4-dimensional, and you output mean and log standard deviation for 4-dimensional action
    x -> (x[1:4], x[5:8])  # Split the output into two parts,
    # the first part is the mean, the second part is the log standard deviation
)
policy_chain = policy_chain |> gpu

Chain(
  Dense(16 => 64, relu),                # 1_088 parameters
  Dense(64 => 64, relu),                # 4_160 parameters
  Dense(64 => 8),                       # 520 parameters
  var"#17#18"(),
)                   # Total: 6 arrays, 5_768 parameters, 856 bytes.

In [24]:
policy = PPOPolicy(env, policy_chain, 0.1, 0.99, 0.1, 0.1, 0.1, 0.1, 0.1, 0.95, 0.1)
### Run the PPO algorithm
policy, total_reward, num_steps, trajectory = PPO(policy, 1000, 32)

In [22]:
for j in 1:1000
	traj = trajectory[j]
	x = [traj[i][1] for i in 1:length(traj)]
	y = [traj[i][2] for i in 1:length(traj)]
	z = [traj[i][3] for i in 1:length(traj)]
	### Plot the trajectory
	plot(x, y, z, label="trajectory", xlabel="x", ylabel="y", zlabel="z", title="Drone Trajectory $j", legend=:bottomright)
	### Save the plot
	savefig("trajectories/trajectory-$j.png")
# plot!(x, y, z)
### Plot the drone
# scatter!([x[end]], [y[end]], [z[end]], label="")
end

LoadError: BoundsError: attempt to access 5-element Vector{Any} at index [6]

In [ ]:
CUDA.version()